In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
def load_data(image_dir, mask_dir, calipers_dir, img_size=(176, 320)):
    
    images, masks, calipers = [], [], []
    
    # Get filenames
    image_files = {os.path.splitext(f)[0]: f for f in os.listdir(image_dir)}
    mask_files = {os.path.splitext(f)[0]: f for f in os.listdir(mask_dir)}
    caliper_files = {os.path.splitext(f)[0]: f for f in os.listdir(calipers_dir)}
    
    # Find common filenames in all folders
    common_keys = set(image_files.keys()) & set(mask_files.keys()) & set(caliper_files.keys())
    
    for key in sorted(common_keys):
        img_path = os.path.join(image_dir, image_files[key])
        mask_path = os.path.join(mask_dir, mask_files[key])
        caliper_path = os.path.join(calipers_dir, caliper_files[key])
        
        try:
            # Load image and mask in grayscale
            img = Image.open(img_path).convert("L")
            mask = Image.open(mask_path).convert("L")
            img = img.resize(img_size)
            mask = mask.resize(img_size)
            
            # Convert to normalized arrays
            img = np.array(img) / 255.0
            mask = np.array(mask) / 255.0
        except Exception as e:
            print(f"Error loading {key}: {e}")
            continue
        
        # Load caliper coordinates
        try:
            caliper = np.load(caliper_path)
        except Exception as e:
            print(f"Error loading caliper {key}: {e}")
            continue

        # Store processed data
        images.append(img)
        masks.append(mask)
        
        # Convert to 1D
        calipers.append(caliper.flatten()) 
    
    # Convert lists to NumPy arrays and reshape accordingly
    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)
    masks = np.array(masks).reshape(-1, img_size[0], img_size[1], 1)
    
    # Concatenate image and mask into a single tensor
    inputs = np.concatenate([images, masks], axis=-1)  
    calipers = np.array(calipers)
    
    return inputs, calipers

In [ ]:
# Load the model from the .keras file
model = load_model(r"C:\Users\Mafe Valenzuela\Documents\Cps\Códigos\calipers_cnn.keras")

In [ ]:
# Computes a confidence interval using the bootstrap resampling method.
# Generates multiple bootstrap samples, computes their means, and determines the confidence interval based on percentiles.
def bootstrap_ci(data, num_samples=1000, confidence=0.95):

    data = np.array(data)
    sample_means = []

    # Generate bootstrap samples and compute the mean for each sample
    for _ in range(num_samples):
        sample = np.random.choice(data, size=len(data), replace=True)  # Resampling with replacement
        sample_means.append(np.mean(sample))

    # Compute percentiles to obtain the confidence interval
    lower_bound = np.percentile(sample_means, (1 - confidence) / 2 * 100)
    upper_bound = np.percentile(sample_means, (1 + confidence) / 2 * 100)
    mean = np.mean(sample_means)

    return mean, lower_bound, upper_bound

In [ ]:
def mean_relative_error(y_true, y_pred):
    # Compute individual relative errors
    mre_values = np.abs((y_true - y_pred) / (y_true + 1e-8)).flatten()
    
    # Compute MRE and its confidence interval using bootstrap
    mean_mre, lower_mre, upper_mre = bootstrap_ci(mre_values)

    return mean_mre, lower_mre, upper_mre


In [ ]:
train_image_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test\images"
train_mask_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test\masks_generates"
train_calipers_dir= r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test\callipers"
train_inputs, train_calipers = load_data(train_image_dir, train_mask_dir, train_calipers_dir)

val_image_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\images"
val_mask_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\masks_generates"
val_calipers_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\callipers"
val_inputs, val_calipers = load_data(val_image_dir, val_mask_dir, val_calipers_dir)

test_image_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test\images"
test_mask_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test\masks_generates"
test_calipers_dir = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\test\callipers"
test_inputs, test_calipers = load_data(test_image_dir, test_mask_dir, test_calipers_dir)

In [ ]:
import numpy as np

def compute_distance_error(original_calipers, predicted_calipers):
    # Compute Euclidean distances for original and predicted calipers
    original_distances = np.linalg.norm(original_calipers[:, :2] - original_calipers[:, 2:], axis=1)
    predicted_distances = np.linalg.norm(predicted_calipers[:, :2] - predicted_calipers[:, 2:], axis=1)

    # Compute absolute and relative errors
    mae = np.mean(np.abs(original_distances - predicted_distances))
    mre = np.mean(np.abs((original_distances - predicted_distances) / (original_distances + 1e-8)))

    return mae, mre, original_distances, predicted_distances

*Calculation of results*

In [ ]:
y_pred_train = model.predict(train_inputs)

mean_mre_train, lower_mre_train, upper_mre_train = mean_relative_error(train_calipers, y_pred_train)
mre_mean_train, mre_lower_train, mre_upper_train = bootstrap_ci(np.abs((original_distances_train - predicted_distances_train) / (original_distances_train + 1e-8)))

print(f"Distance MRE train: {mre_dist_train:.4f} (95% CI: [{mre_lower_train:.4f}, {mre_upper_train:.4f}])")
print(f"Mean Relative Error Train (MRE) (CI:95%): {mean_mre_train:.4f}  ({lower_mre_train:.4f}, {upper_mre_train:.4f})")

In [ ]:
y_pred_val = model.predict(val_inputs)

mean_mre_val, lower_mre_val, upper_mre_val = mean_relative_error(val_calipers, y_pred_val)
mre_mean_val, mre_lower_val, mre_upper_val = bootstrap_ci(np.abs((original_distances_val - predicted_distances_val) / (original_distances_val + 1e-8)))

print(f"Distance MRE validation: {mre_dist_val:.4f} (95% CI: [{mre_lower_val:.4f}, {mre_upper_val:.4f}])")
print(f"Mean Relative Error Validation (MRE) (CI:95%): {mean_mre_val:.4f}  ({lower_mre_val:.4f}, {upper_mre_val:.4f})")

In [ ]:
y_pred_test = model.predict(test_inputs)

mean_mre_test, lower_mre_test, upper_mre_test = mean_relative_error(test_calipers, y_pred_test)
mre_mean_test, mre_lower_test, mre_upper_test = bootstrap_ci(np.abs((original_distances_test - predicted_distances_test) / (original_distances_test + 1e-8)))

print(f"Distance MRE test: {mre_dist_test:.4f} (95% CI: [{mre_lower_test:.4f}, {mre_upper_test:.4f}])")
print(f"Mean Relative Error Test (MRE) (CI:95%): {mean_mre_test:.4f}  ({lower_mre_test:.4f}, {upper_mre_test:.4f})")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def visualize_calipers(image, true_calipers, pred_calipers):
    plt.figure(figsize=(6, 4))
    plt.imshow(image, cmap='gray')  # Mostrar solo la imagen en escala de grises

    # Dibujar los calipers reales (azul)
    plt.plot([true_calipers[0], true_calipers[2]], 
             [true_calipers[1], true_calipers[3]], 
             'bo-', label="Calipers Reales")

    # Dibujar los calipers predichos (rojo)
    plt.plot([pred_calipers[0], pred_calipers[2]], 
             [pred_calipers[1], pred_calipers[3]], 
             'ro-', label="Calipers Predichos")

    plt.legend()
    plt.title("Comparación de Calipers")
    plt.show()

# Seleccionar una imagen de validación
idx = 2  # Cambia este valor para probar diferentes imágenes

image_path = r"C:\Users\Mafe Valenzuela\Documents\Cps\01\Segmentación\val\images\2.png"  
image = mpimg.imread(image_path)  # Cargar la imagen correctamente

true_calipers = val_calipers[idx]  # Calipers reales
pred_calipers = model.predict(val_inputs[idx:idx+1])[0]  # Predicción del modelo

# Visualizar los calipers
visualize_calipers(image, true_calipers, pred_calipers)
